# Tutaj rozpoczyna się proces rozpoznawania działań.

Wczytanie modelu

In [291]:
import numpy as np 
import keras
import tensorflow as tf
import PIL
import cv2 # Biblioteka pozwalająca na odczytywanie poszczególnych partii obrazu.

my_model = keras.saving.load_model('equation_solver_v3.keras', custom_objects=None, compile=True, safe_mode=True)

my_model.summary()

Model: "sequential_7"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ rescaling_3 (Rescaling)         │ (None, 45, 45, 1)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_9 (Conv2D)               │ (None, 45, 45, 16)     │           160 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_9 (MaxPooling2D)  │ (None, 22, 22, 16)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_10 (Conv2D)              │ (None, 22, 22, 32)     │         4,640 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_10 (MaxPooling2D) │ (None, 11, 11, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_11 (Conv2D)              │ (None, 11, 11, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_11 (MaxPooling2D) │ (None, 5, 5, 64)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_3 (Dropout)             │ (None, 5, 5, 64)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_3 (Flatten)             │ (None, 1600)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 128)            │       204,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ outputs (Dense)                 │ (None, 20)             │         2,580 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 692,414 (2.64 MB)

 Trainable params: 230,804 (901.58 KB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 461,610 (1.76 MB)

# Załadowanie nazw klas i parametrów

In [292]:
IMG_WIDTH = 45
IMG_HEIGHT = 45
class_names = ['!', '(', ')', '+', '-', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', '=', '[', ']', 'div', 'times']

# Przykładowe użycie modelu do rozpoznania liczby

In [293]:
img = PIL.Image.open('examples/four.png').convert('L')
img = img.resize((IMG_WIDTH, IMG_HEIGHT))
img_array = keras.utils.img_to_array(img)
img_array = tf.expand_dims(img_array, 0)

predictions = my_model.predict(img_array)
score = tf.nn.softmax(predictions[0])

print(
    "This image most likely belongs to {} with a {:.2f} percent confidence."
    .format(class_names[np.argmax(score)], 100 * np.max(score))
)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 59ms/step
This image most likely belongs to 4 with a 96.31 percent confidence.


:)

# Wyświetlenie obrazu działania matematycznego

In [294]:
image = cv2.imread('examples/8plus8.png', 0)

# Przekonwertowanie obrazu na binarny

In [295]:
binary_image = cv2.adaptiveThreshold(image, 255, cv2.ADAPTIVE_THRESH_MEAN_C, cv2.THRESH_BINARY_INV, 11, 4)

# Znalezienie poszczególnych liczb i znaków na obrazie

In [296]:
contours, _ = cv2.findContours(binary_image, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
contours = sorted(contours, key=lambda x: cv2.boundingRect(x)[0])
contour_image = cv2.cvtColor(image, cv2.COLOR_GRAY2BGR)
for contour in contours:
    x, y, w, h = cv2.boundingRect(contour)
    cv2.rectangle(contour_image, (x, y), (x + w, y + h), (0, 255, 0), 2)

# Filtrowanie szumu z obrazu

In [297]:
filtered_rectangles = []
for contour in contours:
    x, y, w, h = cv2.boundingRect(contour)
    new_rect = (x, y, x + w, y + h)

    overlap = False
    for rect in filtered_rectangles:
        if (new_rect[0] < rect[2] and new_rect[2] > rect[0] and
            new_rect[1] < rect[3] and new_rect[3] > rect[1]):
            overlap = True
            break
    
    if not overlap:
        filtered_rectangles.append(new_rect)

contour_image = cv2.cvtColor(image, cv2.COLOR_GRAY2BGR)
for rect in filtered_rectangles:
    cv2.rectangle(contour_image, (rect[0], rect[1]), (rect[2], rect[3]), (0, 255, 0), 2)

In [298]:
resized_images = []
target_size = (45, 45)

for rect in filtered_rectangles:
    x1, y1, x2, y2 = rect
    roi = image[y1:y2, x1:x2]
    resized_roi = cv2.resize(roi, target_size)
    resized_images.append(resized_roi)

In [299]:
predicted_symbols = []

for resized_image in resized_images:
    resized_image = cv2.resize(resized_image, (45, 45))
    resized_image_normalized = np.expand_dims(resized_image, axis=-1)
    resized_image_normalized = np.expand_dims(resized_image_normalized, axis=0)
    prediction = my_model.predict(resized_image_normalized)
    predicted_class_index = np.argmax(prediction)
    predicted_symbol = class_names[predicted_class_index]
    predicted_symbols.append(predicted_symbol)

predicted_string = ''.join(predicted_symbols)
print("Predicted string:", predicted_string)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 56ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
Predicted string: 8+8
